# **MRI Brain Alzheimer Classification - DINOv2 Model with Adversarial Attacks**

**Clone Dataset from GitHub**

In [ ]:
# Clone repository from GitHub
# !git clone https://github.com/nivbartov/MRI_Brain_Alzheimer_Classification

# %cd MRI_Brain_Alzheimer_Classification
%cd ..

**Import Packages**

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import sys
import os
import numpy as np
import pandas as pd
import torch
from torchsummary import summary
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Subset, ConcatDataset
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from kornia import augmentation as K
from kornia.augmentation import AugmentationSequential
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import random
import time
from datetime import datetime
import json
import gc
import subprocess

**Import Internal Functions**

In [ ]:
from models import def_models
from utils import utils_funcs
from utils import optuna_search

**Device Definition**

In [ ]:
print(f'torch.cuda.is_available(): {torch.cuda.is_available()}')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')
torch.cuda.empty_cache()
gc.collect()
print(torch.cuda.memory_summary())
utils_funcs.open_nvitop()

**Load Dataset**

In [ ]:
train_set = torch.load('dataset/dataset_variables/train_set.pt')
validation_set = torch.load('dataset/dataset_variables/validation_set.pt')
test_set = torch.load('dataset/dataset_variables/test_set.pt')

**Model Definition and Initialization**

In [ ]:
# Initialize input and output sizes
input_width = 224
input_height = 224
input_channels = 3
output_channels = 4

dino_v2_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

# Freeze DINOv2 layers
for param in dino_v2_model.parameters():
    param.requires_grad = False

# Create DINOv2 model with FC layers before the output
model = def_models.DINO_v2_FT(dino_backbone=dino_v2_model, output_channels=output_channels).to(device)

**Data Pre-Processing**

In [ ]:
# Resize images from 128x128 to 224x224
transform = transforms.Compose([
    transforms.Resize((input_height, input_width)),
    transforms.ToTensor()
])

# Apply transformation on the datasets
train_set = utils_funcs.apply_transformations(train_set, transform)
validation_set = utils_funcs.apply_transformations(validation_set, transform)
test_set = utils_funcs.apply_transformations(test_set, transform)

**Model Statistics**

In [ ]:
model_name = type(model).__name__
print("Model", model_name, "statistics:")
summary(model, (input_channels, input_height, input_width))

**Set Hyper-Parameters Using Optuna**

In [ ]:
load_existing_hyperparams = True

if load_existing_hyperparams:
    # Load hyper-parameters (relevant if optuna has already used)
    optuna_params_path = f'./checkpoints/op_tuna_params/{model_name}/DINO_v2_FT_0.9109.json'
    with open(optuna_params_path, 'r') as file:
        hyperparams = json.load(file)
else:
    # Find optimal hyper-parameters using optuna
    hyperparams = optuna_search.optuna_param_search(model_name="DINO_v2_FT", loss_criterion=nn.CrossEntropyLoss(), num_epochs_for_experiments=10, device=device, transfer_learning=True)

In [ ]:
# Set hyper-parameters
batch_size = hyperparams['batch_size']
learning_rate = hyperparams['lr']
optimizer_name = hyperparams['optimizer']
optimizer = getattr(torch.optim, optimizer_name)(model.fc.parameters(), lr=learning_rate)
scheduler = hyperparams['scheduler']
scheduler = StepLR(optimizer, 10, 0.1, verbose=True) if scheduler == "StepLR" else CosineAnnealingLR(optimizer, num_epochs, verbose=True)
num_epochs = 50
criterion = nn.CrossEntropyLoss()

**Create Data Loader**

In [ ]:
# Test loader witch batch size of 1 (in order to perform attacks one by one)
testloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=6, pin_memory=True)

**Load Trained Model**

In [ ]:
pretrained_model_path = f"./checkpoints/{model_name}_074007_01102024.pth"
checkpoint = torch.load(pretrained_model_path,weights_only=True)
last_epoch, last_loss = utils_funcs.load_model(model, optimizer, pretrained_model_path)
model.load_state_dict(checkpoint['net'])

**Perform Fast Gradient Sign (FGSM) Attack**

In [ ]:
accuracies = []
examples = []
epsilons = [0, .0005, .005, .05, .1, .15, .2, .25, .3]

for eps in epsilons:
    acc, ex = utils_funcs.test_single_point_attack(model, device, testloader, 'fgsm', eps)
    accuracies.append(acc)
    examples.append(ex)

utils_funcs.print_accuracy_table(epsilons, accuracies)

In [ ]:
utils_funcs.plot_adversarial_examples(epsilons, examples)

**Perform Projected Gradient Descent (PGD) Attack**

In [ ]:
accuracies = []
examples = []
epsilons = [0, .0005, .005, .05, .1, .15, .2, .25, .3]
alpha = 0.3
num_iter = 5

for eps in epsilons:
    acc, ex = utils_funcs.test_single_point_attack(model, device, testloader, 'pgd', eps, alpha=alpha, num_iter=num_iter)
    accuracies.append(acc)
    examples.append(ex)
    torch.cuda.empty_cache()
    gc.collect()

utils_funcs.print_accuracy_table(epsilons, accuracies)

In [ ]:
utils_funcs.plot_adversarial_examples(epsilons, examples)

**Train Robust Model Against Adversarial Attacks**

In [ ]:
# Set hyper-parameters
num_epochs = 2
epsilon = 0.005
adv_weight = 0.5

# Combine train and validations sets for the full training
full_train_set = ConcatDataset([train_set, validation_set])

# Data loading and shuffling
trainloader = torch.utils.data.DataLoader(full_train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
epoch_losses = utils_funcs.adversarial_train(model, num_epochs, trainloader, device, criterion, optimizer, attack_type='fgsm', epsilon=epsilon, adv_weight=adv_weight, alpha=alpha, num_iter=num_iter)

**Results**

In [ ]:
test_accuracy = utils_funcs.calculate_accuracy(model, testloader, device)
print("Test accuracy on regular test set: {:.3f}%".format(test_accuracy))

test_accuracy_adversial_attk = utils_funcs.calculate_accuracy_attack(model, testloader,device,attack_type='fgsm',epsilon=epsilon)
print("Test accuracy on adversial attacks: {:.3f}%".format(test_accuracy_adversial_attk))

In [ ]:
class_names = ['VeryMildDemented', 'NonDemented', 'ModerateDemented', 'MildDemented']
utils_funcs.plot_normalized_confusion_matrix(testloader, model, class_names, device, model_name)

In [ ]:
utils_funcs.plot_loss_curve(epoch_losses, num_epochs)